# Experiment 007: Exhaustive Search for Small N

For N=2-5, the search space is tractable. We'll do exhaustive/near-exhaustive search over:
- Rotation angles (0-360 degrees)
- Relative positions

This is fundamentally different from SA/GA which can get stuck in local optima.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import json
from tqdm import tqdm
import itertools

getcontext().prec = 30

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        self._update_polygon()
    
    def _update_polygon(self):
        # Tree dimensions
        trunk_w, trunk_h = 0.15, 0.2
        base_w, mid_w, top_w = 0.7, 0.4, 0.25
        tip_y, tier_1_y, tier_2_y, base_y = 0.8, 0.5, 0.25, 0.0
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (0.0, tip_y),
            (top_w/2, tier_1_y), (top_w/4, tier_1_y),
            (mid_w/2, tier_2_y), (mid_w/4, tier_2_y),
            (base_w/2, base_y), (trunk_w/2, base_y),
            (trunk_w/2, trunk_bottom_y), (-trunk_w/2, trunk_bottom_y),
            (-trunk_w/2, base_y), (-base_w/2, base_y),
            (-mid_w/4, tier_2_y), (-mid_w/2, tier_2_y),
            (-top_w/4, tier_1_y), (-top_w/2, tier_1_y),
        ])
        rotated = affinity.rotate(initial_polygon, self.angle, origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)

def has_collision(trees):
    for i in range(len(trees)):
        for j in range(i+1, len(trees)):
            if trees[i].polygon.intersects(trees[j].polygon) and not trees[i].polygon.touches(trees[j].polygon):
                return True
    return False

def get_bounding_box_side(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def calculate_score(trees):
    n = len(trees)
    side = get_bounding_box_side(trees)
    return side ** 2 / n

print('Functions defined.')

Functions defined.


In [2]:
# Load baseline for comparison
def load_solution(csv_path):
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.strip().str.lstrip('s')
    df['y'] = df['y'].astype(str).str.strip().str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    solution = {}
    for group_id, group_data in df.groupby('group_id'):
        n = int(group_id)
        trees = [(row['x'], row['y'], row['deg']) for _, row in group_data.iterrows()]
        solution[n] = trees
    
    return solution

baseline = load_solution('/home/code/experiments/004_cpp_sa_optimizer/input.csv')

# Get baseline scores for small N
for n in range(1, 11):
    trees = [ChristmasTree(x, y, deg) for x, y, deg in baseline[n]]
    score = calculate_score(trees)
    print(f'N={n}: baseline score = {score:.6f}')

N=1: baseline score = 0.661250
N=2: baseline score = 0.450779
N=3: baseline score = 0.434745
N=4: baseline score = 0.416545
N=5: baseline score = 0.416850
N=6: baseline score = 0.399610
N=7: baseline score = 0.399897
N=8: baseline score = 0.385407
N=9: baseline score = 0.387415
N=10: baseline score = 0.376630


In [3]:
def exhaustive_search_n2(angle_step=5, position_step=0.05):
    """
    Exhaustive search for N=2.
    Fix tree 1 at origin with angle 0.
    Search over tree 2's angle and position.
    """
    best_score = float('inf')
    best_config = None
    
    # Tree 1 fixed at origin
    angles_1 = range(0, 360, angle_step)
    angles_2 = range(0, 360, angle_step)
    
    # Position search range (relative to tree 1)
    positions = np.arange(-2, 2, position_step)
    
    total_configs = len(angles_1) * len(angles_2) * len(positions) * len(positions)
    print(f'Searching {total_configs:,} configurations for N=2...')
    
    checked = 0
    for a1 in angles_1:
        for a2 in angles_2:
            for dx in positions:
                for dy in positions:
                    trees = [
                        ChristmasTree('0', '0', str(a1)),
                        ChristmasTree(str(dx), str(dy), str(a2))
                    ]
                    
                    if has_collision(trees):
                        continue
                    
                    score = calculate_score(trees)
                    if score < best_score:
                        best_score = score
                        best_config = [(0, 0, a1), (dx, dy, a2)]
                    
                    checked += 1
        
        # Progress update
        if a1 % 45 == 0:
            print(f'  Angle1={a1}: checked {checked:,} valid configs, best={best_score:.6f}')
    
    print(f'\nN=2 exhaustive search complete.')
    print(f'  Checked {checked:,} valid configurations')
    print(f'  Best score: {best_score:.6f}')
    print(f'  Best config: {best_config}')
    
    return best_config, best_score

# Run exhaustive search for N=2
best_n2_config, best_n2_score = exhaustive_search_n2(angle_step=5, position_step=0.05)

Searching 33,177,600 configurations for N=2...


  Angle1=0: checked 414,863 valid configs, best=0.605000


  Angle1=45: checked 4,148,391 valid configs, best=0.476473


  Angle1=90: checked 7,881,922 valid configs, best=0.476473


  Angle1=135: checked 11,615,465 valid configs, best=0.476473


  Angle1=180: checked 15,348,984 valid configs, best=0.476473


  Angle1=225: checked 19,082,533 valid configs, best=0.476473


  Angle1=270: checked 22,815,984 valid configs, best=0.476473


  Angle1=315: checked 26,549,499 valid configs, best=0.476473



N=2 exhaustive search complete.
  Checked 29,868,097 valid configurations
  Best score: 0.476473
  Best config: [(0, 0, 205), (-0.2999999999999985, -0.5499999999999987, 25)]


In [4]:
# Compare to baseline
baseline_n2_trees = [ChristmasTree(x, y, deg) for x, y, deg in baseline[2]]
baseline_n2_score = calculate_score(baseline_n2_trees)

print(f'\nN=2 comparison:')
print(f'  Baseline score: {baseline_n2_score:.6f}')
print(f'  Exhaustive search score: {best_n2_score:.6f}')
print(f'  Improvement: {baseline_n2_score - best_n2_score:.6f}')

if best_n2_score < baseline_n2_score - 1e-9:
    print('  *** IMPROVEMENT FOUND! ***')
else:
    print('  No improvement (baseline is optimal or better)')


N=2 comparison:
  Baseline score: 0.450779
  Exhaustive search score: 0.476473
  Improvement: -0.025694
  No improvement (baseline is optimal or better)


In [5]:
# Fine-grained search around the best configuration found
def fine_search_n2(initial_config, angle_step=1, position_step=0.01):
    """
    Fine-grained search around a good configuration.
    """
    best_score = float('inf')
    best_config = None
    
    a1_init, a2_init = initial_config[0][2], initial_config[1][2]
    dx_init, dy_init = initial_config[1][0], initial_config[1][1]
    
    # Search around initial angles
    angles_1 = range(int(a1_init) - 10, int(a1_init) + 11, angle_step)
    angles_2 = range(int(a2_init) - 10, int(a2_init) + 11, angle_step)
    
    # Search around initial position
    positions_x = np.arange(dx_init - 0.2, dx_init + 0.2, position_step)
    positions_y = np.arange(dy_init - 0.2, dy_init + 0.2, position_step)
    
    print(f'Fine search around best config...')
    
    for a1 in angles_1:
        for a2 in angles_2:
            for dx in positions_x:
                for dy in positions_y:
                    trees = [
                        ChristmasTree('0', '0', str(a1)),
                        ChristmasTree(str(dx), str(dy), str(a2))
                    ]
                    
                    if has_collision(trees):
                        continue
                    
                    score = calculate_score(trees)
                    if score < best_score:
                        best_score = score
                        best_config = [(0, 0, a1), (dx, dy, a2)]
    
    print(f'Fine search complete. Best score: {best_score:.6f}')
    return best_config, best_score

if best_n2_config is not None:
    fine_n2_config, fine_n2_score = fine_search_n2(best_n2_config)

Fine search around best config...


Fine search complete. Best score: 0.457378


In [ ]:
# Now try N=3 with beam search
def beam_search_n3(beam_width=100, angle_step=10, position_step=0.1):
    """
    Beam search for N=3.
    Start with best N=2 config, add third tree.
    """
    # Start with tree 1 at origin
    angles = range(0, 360, angle_step)
    positions = np.arange(-2, 2, position_step)
    
    # Generate all valid 2-tree configs
    print('Generating 2-tree configurations...')
    two_tree_configs = []
    for a1 in angles:
        for a2 in angles:
            for dx in positions:
                for dy in positions:
                    trees = [
                        ChristmasTree('0', '0', str(a1)),
                        ChristmasTree(str(dx), str(dy), str(a2))
                    ]
                    if not has_collision(trees):
                        score = get_bounding_box_side(trees)
                        two_tree_configs.append(((0, 0, a1), (dx, dy, a2), score))
    
    # Keep top beam_width configs
    two_tree_configs.sort(key=lambda x: x[2])
    two_tree_configs = two_tree_configs[:beam_width]
    print(f'Kept {len(two_tree_configs)} best 2-tree configs')
    
    # Extend to 3 trees
    print('Extending to 3-tree configurations...')
    best_score = float('inf')
    best_config = None
    
    for t1, t2, _ in tqdm(two_tree_configs):
        for a3 in angles:
            for dx in positions:
                for dy in positions:
                    trees = [
                        ChristmasTree(str(t1[0]), str(t1[1]), str(t1[2])),
                        ChristmasTree(str(t2[0]), str(t2[1]), str(t2[2])),
                        ChristmasTree(str(dx), str(dy), str(a3))
                    ]
                    
                    if has_collision(trees):
                        continue
                    
                    score = calculate_score(trees)
                    if score < best_score:
                        best_score = score
                        best_config = [t1, t2, (dx, dy, a3)]
    
    print(f'\nN=3 beam search complete.')
    print(f'  Best score: {best_score:.6f}')
    
    return best_config, best_score

best_n3_config, best_n3_score = beam_search_n3(beam_width=50, angle_step=15, position_step=0.15)

In [ ]:
# Compare N=3 to baseline
baseline_n3_trees = [ChristmasTree(x, y, deg) for x, y, deg in baseline[3]]
baseline_n3_score = calculate_score(baseline_n3_trees)

print(f'\nN=3 comparison:')
print(f'  Baseline score: {baseline_n3_score:.6f}')
print(f'  Beam search score: {best_n3_score:.6f}')
print(f'  Improvement: {baseline_n3_score - best_n3_score:.6f}')

if best_n3_score < baseline_n3_score - 1e-9:
    print('  *** IMPROVEMENT FOUND! ***')
else:
    print('  No improvement (baseline is optimal or better)')

In [ ]:
# Summarize results and save
improvements = []
best_solution = {n: list(baseline[n]) for n in baseline}

# Check N=2
if best_n2_score < baseline_n2_score - 1e-9:
    improvements.append((2, baseline_n2_score, best_n2_score))
    best_solution[2] = [(str(x), str(y), str(a)) for x, y, a in best_n2_config]

# Check N=3
if best_n3_score < baseline_n3_score - 1e-9:
    improvements.append((3, baseline_n3_score, best_n3_score))
    best_solution[3] = [(str(x), str(y), str(a)) for x, y, a in best_n3_config]

print(f'\nSummary:')
print(f'  Improvements found: {len(improvements)}')
for n, old, new in improvements:
    print(f'    N={n}: {old:.6f} -> {new:.6f} (improvement: {old-new:.6f})')

# Calculate total score
def score_config(trees_data):
    tree_list = [ChristmasTree(x, y, deg) for x, y, deg in trees_data]
    return calculate_score(tree_list)

new_total = sum(score_config(best_solution[n]) for n in range(1, 201))
baseline_total = sum(score_config(baseline[n]) for n in range(1, 201))

print(f'\nTotal scores:')
print(f'  Baseline: {baseline_total:.6f}')
print(f'  After exhaustive search: {new_total:.6f}')
print(f'  Improvement: {baseline_total - new_total:.6f}')

In [ ]:
# Save submission
rows = []
for n in range(1, 201):
    for i, (x, y, deg) in enumerate(best_solution[n]):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{x}',
            'y': f's{y}',
            'deg': f's{deg}'
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/submission/submission.csv', index=False)
submission_df.to_csv('submission.csv', index=False)
print(f'Saved submission with {len(submission_df)} rows')

In [ ]:
# Save metrics
metrics = {
    'cv_score': new_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - new_total,
    'n2_baseline': baseline_n2_score,
    'n2_exhaustive': best_n2_score,
    'n3_baseline': baseline_n3_score,
    'n3_beam_search': best_n3_score,
    'improvements': [(n, old, new) for n, old, new in improvements]
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f'Saved metrics')